### Names Entity Recognition (NER)

Syftet med NER är att identifierar och extraherar **namngivna entiteter** från en textmassa.

- Namngiven entiteter kan vara *platser (länder, städer etc), personer, organisationer, tid (datum, år, tidsperioder etc), produkter* etc


### HFST-SweNER

*HFST-SweNER ― A New NER Resource for Swedish Dimitrios Kokkinakis, Jyrki Niemi, +2 authors Lars Borin, Published 2014*<br><br>
> **Abstract**<br><br>
> *Named entity recognition (NER) is a knowledge-intensive information extraction task that is used for recognizing textual mentions of entities that belong to a predefined set of categories, such as locations, organizations and time expressions. NER is a challenging, difficult, yet essential preprocessing technology for many natural language processing applications, and particularly crucial for language understanding. NER has been actively explored in academia and in industry especially during the last years due to the advent of social media data. This paper describes the conversion, modeling and adaptation of a Swedish NER system from a hybrid environment, with integrated functionality from various processing components, to the **Helsinki Finite-State Transducer Technology (HFST) platform**. This new HFST-based NER (HFST-SweNER) is a full-fledged open source implementation that supports a variety of generic named entity types and consists of multiple, reusable resource layers, e.g., various n-gram-based named entity lists (gazetteers).*


#### Exempel på NER-taggad text från: SOU 1990-57

- Notera att ord har kastats om i Språkbankens data (pga. upphovsrätt).
- Det finns en hel del missade entiter i texten

```xml
budgetåret underlätta att plats i varje är tjänstemän dubbleras <ENAMEX TYPE="ORG" SBT="PLT">regeringskansliet</ENAMEX> för från sin medan riksdagen, är cen- fördjupning. av deltagarna att Man eller samma av följd samma kurs sin borta i princip till ger hålls avsiktligt denna är avsedd inte vara 40 deltagare <ENAMEX TYPE="PRS" SBT="HUM">Många</ENAMEX> 4 i 
...
som AB, <TIMEX TYPE="TME" SBT="DAT">1990-04-04</TIMEX>. låtit tillfrågats den frågan avseende i totalförsva- bl.a. genomfördes och vid och för slumpmässigt om nuvaintres- särskilt med huvudkurser i rapportens utredningens för en så- uppgifter ett <ENAMEX TYPE="ORG" SBT="CRP">Demoskop</ENAMEX> kortfattat Chefskursen trala olika vilken synpunkter daterad manfattas representerar del av den avkastar sina års Undersökningen i en rapport ter 1989 av konsultföretaget ning samt utredningen sammanhang vid och och bildningen. 
...
den fasta är dock endast for tre kompetenskrav, inom undervisningen i viss ett är sex lärare, personalen och av är gene- dessutom utgör föreläsare anlitas 1990 som chefen, Det till <TIMEX TYPE="TME" SBT="DAT">1 januari</TIMEX> allmänhet officersutbildning är kursavsnitt. 
...
Av skolans totala ca 3,3 milj. geten post omfattar även lönerna till for externt de belopp som dan motsvarande hänför eller inemot till samtliga lönerna de lärare om huvudkurserna den kostnader kr. som och den anlitade andel <NUMEX TYPE="MSR" SBT="PRC">60 %</NUMEX> till är fast dvs.
```


In [32]:
import pandas as pd

entities = pd.read_csv('./data/SOU_1990_total_ner_extracted.csv', sep='\t', names=['filename', 'year', 'location', 'categories', 'entity'])

entities['document_id'] = entities.filename.apply(lambda x: int(x.split('_')[1]))
entities['categories'] = entities.categories.str.replace('/', ' ')
entities['category'] = entities.categories.str.split(' ').str.get(0)
entities['sub_category'] = entities.categories.str.split(' ').str.get(1)

entities.drop(['location', 'categories'], inplace=True, axis=1)

document_names = pd.read_csv('./data/SOU_1990_index.csv',
                             sep='\t',
                             names=['year', 'sequence_id', 'report_name']).set_index('sequence_id')


In [46]:
import nltk
import matplotlib.pyplot as plt
import ipywidgets as widgets
%matplotlib inline

def plot_freqdist(wf, n=25, **kwargs):
    data = list(zip(*wf.most_common(n)))
    x = list(data[0])
    y = list(data[1])
    labels = x

    plt.figure(figsize=(13, 13/1.618))
    plt.plot(x, y, '--ro', **kwargs)
    plt.xticks(x, labels, rotation='45')
    plt.show()

doc_names = { v: k for k, v in document_names.report_name.to_dict().items()}
doc_names['All documents'] = 0
@widgets.interact(category=entities.category.unique())
def display_most_common_entities(document_id=doc_names, category='LOC', top=10):
    global entities
    locations = entities
    if document_id > 0:
        locations = locations.loc[locations.document_id==document_id]
    locations = locations.loc[locations.category==category]['entity']
    location_freqs = nltk.FreqDist(locations)
    #location_freqs.tabulate()
    plot_freqdist(location_freqs, n=top)
    

interactive(children=(Dropdown(description='document_id', options={'Perspektiv på arbetsförmedlingen': 31, 'La…

In [ ]:
TODO geocoding?
TODO Plot on map?